In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
import json
import pickle

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [4]:
import speech_recognition as sr 
import pyttsx3  
import datetime
import webbrowser
import wikipedia

In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
words.extend("")
documents.append(([], "noanswer"))
classes.append("noanswer")

In [8]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

28 documents
8 classes ['Age', 'Joke', 'Story_telling', 'goodbye', 'greeting', 'noanswer', 'options', 'thanks']
53 unique lemmatized words ["'s", ',', '.', 'a', 'any', 'anyone', 'are', 'awesome', 'be', 'birthday', 'bye', 'can', 'chatting', 'could', 'crack', 'day', 'do', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'is', 'joke', 'later', 'me', 'next', 'nice', 'offered', 'okay', 'old', 'provide', 'see', 'story', 'support', 'tell', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'when', 'you', 'your']


In [9]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
6/6 [==============================] - 0s 867us/step - loss: 2.1234 - accuracy: 0.0714
Epoch 2/200
6/6 [==============================] - 0s 1ms/step - loss: 2.1230 - accuracy: 0.1786
Epoch 3/200
6/6 [==============================] - 0s 1ms/step - loss: 1.9402 - accuracy: 0.2500
Epoch 4/200
6/6 [==============================] - 0s 1ms/step - loss: 1.9486 - accuracy: 0.2143
Epoch 5/200
6/6 [==============================] - 0s 1ms/step - loss: 1.7966 - accuracy: 0.5000
Epoch 6/200
6/6 [==============================] - 0s 1ms/step - loss: 1.8318 - accuracy: 0.2143
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 1.6724 - accuracy: 0.5000
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.6394 - accuracy: 0.3929
Epoch 9/200
6/6 [==============================] - 0s 1ms/step - loss: 1.5704 - accuracy: 0.5000
Epoch 10/200
6/6 [==============================] - 0s 997us/step - loss: 1.4926 - accuracy: 0.5714
Epoch 11/200
6/6 [=======

6/6 [==============================] - 0s 1ms/step - loss: 0.1132 - accuracy: 0.9286
Epoch 84/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1184 - accuracy: 0.9643
Epoch 85/200
6/6 [==============================] - 0s 997us/step - loss: 0.1273 - accuracy: 1.0000
Epoch 86/200
6/6 [==============================] - 0s 830us/step - loss: 0.1250 - accuracy: 0.9643
Epoch 87/200
6/6 [==============================] - 0s 1ms/step - loss: 0.1551 - accuracy: 0.9643
Epoch 88/200
6/6 [==============================] - 0s 833us/step - loss: 0.1718 - accuracy: 0.9286
Epoch 89/200
6/6 [==============================] - 0s 998us/step - loss: 0.1174 - accuracy: 0.9286
Epoch 90/200
6/6 [==============================] - 0s 505us/step - loss: 0.1365 - accuracy: 0.9643
Epoch 91/200
6/6 [==============================] - 0s 1ms/step - loss: 0.0782 - accuracy: 1.0000
Epoch 92/200
6/6 [==============================] - 0s 665us/step - loss: 0.1168 - accuracy: 0.9643
Epoch 93/200
6/6 [===

6/6 [==============================] - 0s 831us/step - loss: 0.0301 - accuracy: 1.0000
Epoch 165/200
6/6 [==============================] - 0s 665us/step - loss: 0.0619 - accuracy: 0.9643
Epoch 166/200
6/6 [==============================] - 0s 994us/step - loss: 0.0435 - accuracy: 1.0000
Epoch 167/200
6/6 [==============================] - 0s 665us/step - loss: 0.1085 - accuracy: 0.9286
Epoch 168/200
6/6 [==============================] - 0s 826us/step - loss: 0.0467 - accuracy: 1.0000
Epoch 169/200
6/6 [==============================] - 0s 831us/step - loss: 0.0726 - accuracy: 0.9643
Epoch 170/200
6/6 [==============================] - 0s 665us/step - loss: 0.0614 - accuracy: 0.9643
Epoch 171/200
6/6 [==============================] - 0s 831us/step - loss: 0.0157 - accuracy: 1.0000
Epoch 172/200
6/6 [==============================] - 0s 671us/step - loss: 0.0552 - accuracy: 1.0000
Epoch 173/200
6/6 [==============================] - 0s 831us/step - loss: 0.0446 - accuracy: 1.0000
Epoc

In [11]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [12]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.70
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [13]:
r = sr.Recognizer()

In [14]:
def SpeakText(command): 
      
    # Initialize the engine 
    engine = pyttsx3.init() 
    engine.say(command)  
    engine.runAndWait() 

#https://www.geeksforgeeks.org/python-gui-tkinter/#:~:text=Out%20of%20all%20the%20GUI,tkinter%20is%20an%20easy%20task.

In [15]:
now = datetime.datetime.now()

In [16]:
def googleIt(msg):
    if msg=='time':
        res = now.strftime("The time is %H:%M")
        
    elif 'weather' in msg:
        webbrowser.open_new('https://weather.com/en-IN/weather/today')
        res = 'Click on the new tab to view the weather in your location'
        
    else:
        try:
            res = wikipedia.summary(wikipedia.search(msg)[0], sentences=2)
        except:
            webbrowser.open_new('www.google.com/search?q=' + msg)
            res = 'Googled it for you :)'
    return res

In [17]:
#Creating GUI with tkinter
import tkinter
from tkinter import *

def printChat(msg):
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#303744", font=("Helvetica", 12 ))

        res = chatbot_response(msg)
        
        
        if res=='Not sure I understand':
            res = googleIt(msg)
        
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        SpeakText(res)


def record():
    
    with sr.Microphone() as source2:
        r.adjust_for_ambient_noise(source2, duration=0.2)
        audio2 = r.listen(source2) 
        MyText = r.recognize_google(audio2)
        msg = MyText.lower()

        printChat(msg)
        
        
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    printChat(msg)


base = Tk()
base.title("Rob Oto")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="#eff3f5", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set


#Create Button to send message 
SendButton = Button(base, font=("Helvetica",11,'bold italic'), text="Send", width="6", height=1,
                    bd=0, bg="#5e8ad4", activebackground="#5080d1",fg='#ffffff', command= send )

#Create Button to Record Message
RecordButton = Button(base, font=("Helvetica",11,'bold italic'), text="Rec", width="6", height=1,
                    bd=0, bg="#0ec6b3", activebackground="#0db6a5",fg='#ffffff',
                    command= record )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="#dae4e8",width="29", height="5", font="Helvetica")

#Place all components on the screen
scrollbar.place(x=380,y=6, height=430)
ChatLog.place(x=6,y=6, height=430, width=370)
EntryBox.place(x=60, y=441, height=51, width=278)
SendButton.place(x=6, y=441, height=51, width=50)
RecordButton.place(x=342, y=441, height=51, width=50)


base.mainloop()